<a href="https://colab.research.google.com/github/KainK999/113-2_generative_ai/blob/main/HW07_%E6%88%91%E7%9A%84%E5%B0%8D%E8%A9%B1%E6%A9%9F%E5%99%A8%E4%BA%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Ollama](https://ollama.com/) 可以讓我們在自己的機器上跑開源的大型語言模型, 並且用 API 的方式呼叫。這裡我們介紹在 Colab 上跑, 並且分別用 OpenAI 的 API, 及 [`aisuite` 套件](https://github.com/andrewyng/aisuite) 來使用 Ollama 提供的大型語言模型。

### 1. 安裝並執行 Ollama

首先是到官網抓下安裝程式, 並且執行。

In [ ]:
!curl -fsSL https://ollama.ai/install.sh | sh  #127.0.0.1 -> localhost

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


****因為我們要用 API 的方式呼叫, 所以需要跑 Ollama Server, 這裡我們要求放在背景執行。 (我們需要繼續使用Ollama在Colab中)

In [ ]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [ ]:
# !ollama pull gemma3:1b    #run -> pull
# !ollama pull mistral-openorca:7b
!ollama pull qwen2.5

pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling 2bada8a74506... 100% ▕▏ 4.7 GB                         
pulling 66b9ea09bd5b... 100% ▕▏   68 B                         
pulling eb4402837c78... 100% ▕▏ 1.5 KB                         
pulling 832dd9e00a68... 100% ▕▏  11 KB                         
pulling 2f15b3218f05... 100% ▕▏  487 B                         
verifying sha256 digest 
writing manifest 
success 


### 2. 用 OpenAI API 使用

因為 ChatGPT 大概是最早紅的大型語言模型, 因此許多大型語言模型, 都和 OpenAI API 相容, Ollama 也不例外。

In [ ]:
import openai
from openai import OpenAI

本來是需要 OpenAI 金鑰, 但我們沒有真的要用 OpenAI 的服務, 金鑰就亂打一通就好。

In [ ]:
api_key = "ollama"      #我們在自己的電腦上(Colab)跑得

如同一般 OpenAI API 打開 `client` 的方式, 只是這裡多了 API 服務的網址。注意在自己家 (事實上是 Google Colab 的機器), 預設服務 `port` 是 `11434`。

In [ ]:
client = OpenAI(
    api_key=api_key,
    base_url="http://localhost:11434/v1"
)   #仍需接收api_key的輸入
#v1:ollama 呼叫api的網址

### 6. 打造一個你的對話機器人 web app!

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr

對話機器人 app 設定

In [ ]:
title = "健身教練"
# system = """你是一位健身教練型的對話機器人，說話方式像是熟到會互嘴的老朋友，會用激將法鼓勵對方，但絕不傷人。
# 語氣可以有點討人厭、忠言逆耳、帶點挑釁，但本質是溫暖的，會幫助使用者建立信心。每次回應盡量不要超過二十個字，要像台灣人平常互相鼓勵又嘴砲的口氣。"""
system = "你是一位嘴砲健身教練型的對話機器人，講話像台灣那種罵你罵到醒的壞朋友。你會用激將法、講幹話、挖苦、吐槽來激勵使用者，讓他們被罵到想證明自己。語氣可以毒舌、靠北、有點賤嘴，但不能真的傷人。你的目的就是激起鬥志，讓人咬牙繼續撐下去。每次回應不超過二十字，要像台灣朋友互嘴時那樣嘴到心坎裡。"
description = "我是不會放任你爛掉的健身教練，有事情就跟我聊聊！"
model = "mistral-openorca:7b"

In [ ]:
initial_messages = [{"role":"system",
             "content":system},
            {"role":"assistant",
            'content':description}]

In [ ]:
state = gr.State(initial_messages)  #send initial messages to gradio (in the early days)

In [ ]:
def trainer(prompt, messages):
    messages.append({"role": "user", "content": prompt})
    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model,
        )
    reply = chat_completion.choices[0].message.content
    messages.append({"role": "assistant", "content": reply})
    #history = history + [[prompt, reply]]   #back when openai API and gradio have different format
    return messages, messages

In [ ]:
chatbot = gr.Chatbot(type="messages")

In [ ]:
with gr.Blocks(title=title) as demo:    #要示範的區塊的樣子
    gr.Markdown(f"## 🤖 {title}\n{description}")
    chatbot = gr.Chatbot(type="messages")   #使用Open AI API
    msg = gr.Textbox(label="輸入訊息")
    state = gr.State(initial_messages.copy())  # 務必用 copy() 希望中間被改動時 initial message 不備改動

    msg.submit(fn=trainer, inputs=[msg, state], outputs=[chatbot, state])

demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://70f6c19fb060a9c268.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
